# Proje

Bu projeyi veribilimi ve makine öğrenmesi alanında öğrendiklerimi kendi üzerimde uygulamak için yapıyorum.

## Amaç

* Verisetini işlenebilir hale getirdikten sonra makineye öğretip; hangi saatlerde daha verimli okuma yaptığımı, hangi saatlerde yazdığım özetin sayfa sayısına oranla daha uzun olduğunu, hangi kitapları beğenip beğenmediğimi, en çok hangi yazarların kitaplarında vakit geçirdiğimi, önceki verilere dayanarak saat bilgisi eksik olan okumalarımın saatlerinin ne olabileceği ve daha birçok soruyu veri bilimi ve makine öğrenmesi kullanarak bilgisayarın cevaplamasını sağlamak.

# Veriseti Hikayesi

Bu veriseti, benim kitap okurken çeşitli verileri defterime kaydedip sonra da excele almama dayanır.

* date = Okumanın tarihi

* start_time = Okumaya başladığım saat

* finish_time = Okumayı bitirdiğim saat

* writer = Okuduğum kitabın yazarı

* book = Kitabın adı

* start_page = Okumaya başladığım sayfanın sayısı

* finish_page = Okumayı bitidiğim sayfanın sayısı

* summary_size = Okuduğum sayfa aralıklarını değerlendirip yazdığım özet ya da çıkarımların kelime sayısı

* is_section_liked = Okuduğum aralığı beğendim mi ? 0=Hayır, 1=Evet



In [5]:
import pandas as pd

In [6]:
my_readings = pd.read_excel("My_readings.xlsx")
df = my_readings.copy()

In [6]:
df.head() #veriseti hakkinda ilk 5 gozlem

,date,start_time,finish_time,writer,book,kind,start_page,finish_page,summary_size,is_section_liked
0,2020-05-03,00:00:00,00:00:00,J.P. Sartre,Bulanti,felsefik,15.0,38.0,68.0,1
1,2020-03-09,00:00:00,00:00:00,J.P. Sartre,Bulanti,felsefik,39.0,75.0,73.0,1
2,2020-03-11,00:00:00,00:00:00,J.P. Sartre,Bulanti,felsefik,75.0,105.0,50.0,1
3,2020-03-11,00:00:00,00:00:00,Thomas More,Utopya,utopik,3.0,26.0,91.0,1
4,2020-03-22,00:00:00,00:00:00,J.P. Sartre,Bulanti,felsefik,106.0,134.0,27.0,1


In [7]:
df.shape #verisetimizin gözlem ve değişken sayisi

(35, 10)

In [8]:
df.dtypes #verisetimizdeki değişkenlerin türü

date                datetime64[ns]
start_time                  object
finish_time                 object
writer                      object
book                        object
kind                        object
start_page                 float64
finish_page                float64
summary_size               float64
is_section_liked             int64
dtype: object

In [7]:
# is_section_liked değişkenini integerdan booleana çevirelim
df['is_section_liked'] = df['is_section_liked'].astype('bool')

In [10]:
df.dtypes

date                datetime64[ns]
start_time                  object
finish_time                 object
writer                      object
book                        object
kind                        object
start_page                 float64
finish_page                float64
summary_size               float64
is_section_liked              bool
dtype: object

--

#### Değerler arası tip dönüşümlerine devam etmem gerekiyor ancak eksik veri varken yapmıyor, bu sebeple tüm eksik veriler yerine -1 yazacağım, saat verisi olarak boş olanlara sonradan 00:00:00 bastığımda hata alıyordum bu yüzden excelden yazdım tek tek :(.

In [53]:
# Eğer excelde kendim 00 diye yazmasaydım bu kodları kullanacaktık.

## df["start_time"].fillna("00:00:00", inplace = True)
## df["finish_time"].fillna("00:00:00", inplace = True)

In [9]:
df.fillna(-1, inplace = True)

In [14]:
df.head()

,date,start_time,finish_time,writer,book,kind,start_page,finish_page,summary_size,is_section_liked
0,2020-05-03,00:00:00,00:00:00,J.P. Sartre,Bulanti,felsefik,15.0,38.0,68.0,True
1,2020-03-09,00:00:00,00:00:00,J.P. Sartre,Bulanti,felsefik,39.0,75.0,73.0,True
2,2020-03-11,00:00:00,00:00:00,J.P. Sartre,Bulanti,felsefik,75.0,105.0,50.0,True
3,2020-03-11,00:00:00,00:00:00,Thomas More,Utopya,utopik,3.0,26.0,91.0,True
4,2020-03-22,00:00:00,00:00:00,J.P. Sartre,Bulanti,felsefik,106.0,134.0,27.0,True


Eksik değerleri doldurduk, şimdi dönüşümlere devam edebiliriz.

In [10]:
# start_page, finish_page ve summary_size  değişkenlerini integer yapmak istiyorum
df['start_page'] = df['start_page'].astype('int')
df['finish_page'] = df['finish_page'].astype('int')
df['summary_size'] = df['summary_size'].astype('int')
df.dtypes

date                datetime64[ns]
start_time                  object
finish_time                 object
writer                      object
book                        object
kind                        object
start_page                   int64
finish_page                  int64
summary_size                 int64
is_section_liked              bool
dtype: object

--

Şimdi de writer, kind ve book değişkenlerinin kategorik değişken (category) olmasını istiyorum.

In [11]:
#üstte yaptığımızın aksine bu sefer tek tek değil de hepsini bir satır kodda yapalım.

df[['writer', 'kind', 'book']] = df[['writer', 'kind', 'book']].astype('category')
df.dtypes

date                datetime64[ns]
start_time                  object
finish_time                 object
writer                    category
book                      category
kind                      category
start_page                   int64
finish_page                  int64
summary_size                 int64
is_section_liked              bool
dtype: object

#### Şimdi, okumayı bitirdiğim sayfa numarasından, başladığımı çıkarıp kaç sayfa okuduğumu gösteren bir değişken oluşturmak istiyorum.

In [12]:
read_pages = df["finish_page"] - df["start_page"]
read_pages

0     23
1     36
2     30
3     23
4     28
5     24
6     18
7     21
8      9
9     11
10     4
11     9
12    11
13    14
14    18
15    26
16    10
17    27
18     0
19     0
20     0
21     2
22    35
23    29
24    46
25    57
26    14
27    43
28    36
29    23
30    43
31    37
32    19
33    16
34    46
dtype: int64

In [13]:
# Şimdi bulduğumuz bu sayıları "read_pages" adı altında yeni bir kolon olarak ekleyelim

df["read_pages"] = read_pages

In [61]:
df.head()

,date,start_time,finish_time,writer,book,kind,start_page,finish_page,summary_size,is_section_liked,read_pages
0,2020-05-03,00:00:00,00:00:00,J.P. Sartre,Bulanti,felsefik,15,38,68,True,23
1,2020-03-09,00:00:00,00:00:00,J.P. Sartre,Bulanti,felsefik,39,75,73,True,36
2,2020-03-11,00:00:00,00:00:00,J.P. Sartre,Bulanti,felsefik,75,105,50,True,30
3,2020-03-11,00:00:00,00:00:00,Thomas More,Utopya,utopik,3,26,91,True,23
4,2020-03-22,00:00:00,00:00:00,J.P. Sartre,Bulanti,felsefik,106,134,27,True,28


In [20]:
# okuduğum sayfanın istatistiki özelliklerine bakalım

df["read_pages"].describe()

count    35.000000
mean     22.514286
std      14.691677
min       0.000000
25%      11.000000
50%      23.000000
75%      32.500000
max      57.000000
Name: read_pages, dtype: float64

------

Şimdi finish_time dan start_time ı çıkararak kaç dakika okuduğumu bulmak istiyorum.

In [14]:
df["finish_time"] - df["start_time"]

# Ama datetime.time objeleri arasında çıkartma işlemi yapmıyormuş.

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'

------
Bahsettiğim çıkartmayı yapabilmek için başta tipi datetime.time olan "start_time" ve "finish_time"ı datetime'a çevirmem gerektiğini düşündüm niye böyle düşündüm bilmiyorum ancak hiç gerek yokmuş yine de hatamı silmiyorum kalsın ders alalım :)

In [3]:
# Bahsettiğim hatam

# Şimdi de start_time ve finish_time değişkenlerini object'ten datetime'a çevirelim.
df['start_time'] = pd.to_datetime(df['start_time'])

NameError: name 'df' is not defined

---

Sonrasında şu combine fonksiyonunu buldum, datetime.time'ı tek başına birbirinden çıkaramıyormuşuz, ilk başta gün ve saat olarak birleştirmek gerekiyor ve saat olarak değerleri kaymasın diye ikisine de, sonradan çıkarırken birbirlerini sıfırlasın diye aynı gün tarihini veriyoruz (date.today()).

In [25]:
from datetime import datetime

datetime.combine(date.today(), df["finish_time"]) - datetime.combine(date.today(), df["start_time"])

TypeError: combine() argument 2 must be datetime.time, not Series

 ---
df["finish_time"] ya da df["start_ time"] diyerek direkt olarak dizi olan, dataframe sütunu ile işlem yapmaya kalktık kabul etmedi, bu yüzden dataframein o sütunlarının tek tek satırlarıyla işlem yapmak gerekiyor yani df[finish_time][5] - df[start_time][5] gibi. Bunu da aşağıdaki gibi bir fonksiyonu stackoverflow'dan bir hayırseverin yardımıyla hallettim.

---

In [95]:
def reading_time(df):
       return datetime.combine(date.today(), df["finish_time"]) - datetime.combine(date.today(), df["start_time"])

df['reading_time'] = df.apply(reading_time, axis=1)

In [78]:
df.tail() # görüldüğü üzere time_between olarak kaç dk kitap okuduğum bilgisini ekledik ancak bunun tam sayı olarak dakika cinsinden olmasını istiyorum.

,date,start_time,finish_time,writer,book,kind,start_page,finish_page,summary_size,is_section_liked,read_pages,time_between,reading_time
30,2020-06-15,16:00:00,17:00:00,Dostoyevski,Suc ve Ceza,roman,253,296,11,True,43,60.0,01:00:00
31,2020-06-16,21:00:00,22:00:00,Dostoyevski,Suc ve Ceza,roman,297,334,26,True,37,60.0,01:00:00
32,2020-06-16,22:15:00,22:45:00,Platon,Sokratesin Savunmasi,felsefik,40,59,31,True,19,30.0,00:30:00
33,2020-06-17,16:00:00,16:30:00,Platon,Sokratesin Savunmasi,felsefik,60,76,40,True,16,30.0,00:30:00
34,2020-06-17,21:45:00,22:55:00,Dostoyevski,Suc ve Ceza,roman,335,381,38,True,46,70.0,01:10:00


In [96]:
df["reading_time"] = df["reading_time"].dt.seconds.astype('int')
df.tail()

,date,start_time,finish_time,writer,book,kind,start_page,finish_page,summary_size,is_section_liked,read_pages,time_between,reading_time
30,2020-06-15,16:00:00,17:00:00,Dostoyevski,Suc ve Ceza,roman,253,296,11,True,43,60.0,3600
31,2020-06-16,21:00:00,22:00:00,Dostoyevski,Suc ve Ceza,roman,297,334,26,True,37,60.0,3600
32,2020-06-16,22:15:00,22:45:00,Platon,Sokratesin Savunmasi,felsefik,40,59,31,True,19,30.0,1800
33,2020-06-17,16:00:00,16:30:00,Platon,Sokratesin Savunmasi,felsefik,60,76,40,True,16,30.0,1800
34,2020-06-17,21:45:00,22:55:00,Dostoyevski,Suc ve Ceza,roman,335,381,38,True,46,70.0,4200


Bu şekilde yapınca saniye cinsinden oldu bu yüzden bir aşağıda 60a bölelim dakika olsun.

In [97]:
df["reading_time"] = df["reading_time"]/60
df.tail()

,date,start_time,finish_time,writer,book,kind,start_page,finish_page,summary_size,is_section_liked,read_pages,time_between,reading_time
30,2020-06-15,16:00:00,17:00:00,Dostoyevski,Suc ve Ceza,roman,253,296,11,True,43,60.0,60.0
31,2020-06-16,21:00:00,22:00:00,Dostoyevski,Suc ve Ceza,roman,297,334,26,True,37,60.0,60.0
32,2020-06-16,22:15:00,22:45:00,Platon,Sokratesin Savunmasi,felsefik,40,59,31,True,19,30.0,30.0
33,2020-06-17,16:00:00,16:30:00,Platon,Sokratesin Savunmasi,felsefik,60,76,40,True,16,30.0,30.0
34,2020-06-17,21:45:00,22:55:00,Dostoyevski,Suc ve Ceza,roman,335,381,38,True,46,70.0,70.0


In [99]:
# Float olan reading_time'ı integer'a çevirelim

df['reading_time'] = df['reading_time'].astype('int')
df.tail()

,date,start_time,finish_time,writer,book,kind,start_page,finish_page,summary_size,is_section_liked,read_pages,time_between,reading_time
30,2020-06-15,16:00:00,17:00:00,Dostoyevski,Suc ve Ceza,roman,253,296,11,True,43,60.0,60
31,2020-06-16,21:00:00,22:00:00,Dostoyevski,Suc ve Ceza,roman,297,334,26,True,37,60.0,60
32,2020-06-16,22:15:00,22:45:00,Platon,Sokratesin Savunmasi,felsefik,40,59,31,True,19,30.0,30
33,2020-06-17,16:00:00,16:30:00,Platon,Sokratesin Savunmasi,felsefik,60,76,40,True,16,30.0,30
34,2020-06-17,21:45:00,22:55:00,Dostoyevski,Suc ve Ceza,roman,335,381,38,True,46,70.0,70
